In [3]:
# Full Optimized Code for Bird Classification
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress AVX warnings
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'  # Mitigate OpenMP conflict

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2L
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import numpy as np

In [5]:
# Enhanced Data Augmentation with Mixup and CutMix
class AugmentedGenerator(tf.keras.utils.Sequence):
    def __init__(self, generator, alpha=0.4, cutmix_prob=0.5):
        self.generator = generator
        self.alpha = alpha
        self.cutmix_prob = cutmix_prob

    def __len__(self):
        return len(self.generator)

    def __getitem__(self, index):
        images, labels = self.generator[index]
        
        if np.random.rand() < self.cutmix_prob:
            return self.cutmix(images, labels)
        else:
            return self.mixup(images, labels)
    
    def mixup(self, images, labels):
        indices = np.random.permutation(images.shape[0])
        shuffled_images = images[indices]
        shuffled_labels = labels[indices]
        lam = np.random.beta(self.alpha, self.alpha)
        mixed_images = lam * images + (1 - lam) * shuffled_images
        mixed_labels = lam * labels + (1 - lam) * shuffled_labels
        return mixed_images, mixed_labels

    def cutmix(self, images, labels):
        indices = np.random.permutation(images.shape[0])
        shuffled_images = images[indices]
        shuffled_labels = labels[indices]
        lam = np.random.beta(self.alpha, self.alpha)
        bbx1, bby1, bbx2, bby2 = self.rand_bbox(images.shape, lam)
        images[:, bbx1:bbx2, bby1:bby2, :] = shuffled_images[:, bbx1:bbx2, bby1:bby2, :]
        lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (images.shape[1] * images.shape[2]))
        labels = lam * labels + (1 - lam) * shuffled_labels
        return images, labels

    def rand_bbox(self, size, lam):
        W = size[1]
        H = size[2]
        cut_rat = np.sqrt(1. - lam)
        cut_w = int(W * cut_rat)
        cut_h = int(H * cut_rat)
        cx = np.random.randint(W)
        cy = np.random.randint(H)
        bbx1 = np.clip(cx - cut_w // 2, 0, W)
        bby1 = np.clip(cy - cut_h // 2, 0, H)
        bbx2 = np.clip(cx + cut_w // 2, 0, W)
        bby2 = np.clip(cy + cut_h // 2, 0, H)
        return bbx1, bby1, bbx2, bby2


In [6]:
# Dataset paths (UPDATE THESE TO YOUR LOCAL PATHS)
train_path = 'D:/Dataset/Snakes/Train'
test_path = 'D:/Dataset/Snakes/Test'

In [7]:

# Data Generators with Validation Split
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet_v2.preprocess_input,
    rotation_range=60,
    width_shift_range=0.4,
    height_shift_range=0.4,
    shear_range=0.4,
    zoom_range=0.4,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.4, 1.6],
    validation_split=0.2  # 80% training, 20% validation
)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(480, 480),
    batch_size=4,  # Reduced batch size for memory stability
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(480, 480),
    batch_size=4,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

test_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet_v2.preprocess_input
)
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(480, 480),
    batch_size=4,
    class_mode='categorical',
    shuffle=False
)


Found 63 images belonging to 9 classes.
Found 9 images belonging to 9 classes.
Found 18 images belonging to 9 classes.


In [8]:
# Build Enhanced Model Architecture
base_model = EfficientNetV2L(
    include_top=False,
    weights='imagenet',
    input_shape=(480, 480, 3)
)
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dense(2048, activation='swish', kernel_regularizer='l2'),
    Dropout(0.5),
    Dense(1024, activation='swish', kernel_regularizer='l2'),
    Dropout(0.4),
    Dense(9, activation='softmax')  # Ensure this matches your class count
])

In [9]:
# COMPILE THE MODEL BEFORE TRAINING
optimizer = Adam(learning_rate=1e-4, amsgrad=True)
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)


In [10]:
# First Training Phase (Frozen Backbone)
history = model.fit(
    AugmentedGenerator(train_generator),
    validation_data=val_generator,
    epochs=50
)


Epoch 1/50
16/16 [==============================] - 101s 4s/step - loss: 31.6307 - accuracy: 0.1587 - val_loss: 31.1202 - val_accuracy: 0.4444
Epoch 2/50
16/16 [==============================] - 63s 4s/step - loss: 30.7728 - accuracy: 0.3651 - val_loss: 30.7201 - val_accuracy: 0.5556
Epoch 3/50
16/16 [==============================] - 64s 4s/step - loss: 30.1890 - accuracy: 0.6190 - val_loss: 30.3529 - val_accuracy: 0.5556
Epoch 4/50
16/16 [==============================] - 66s 4s/step - loss: 29.8643 - accuracy: 0.5556 - val_loss: 29.9499 - val_accuracy: 0.6667
Epoch 5/50
16/16 [==============================] - 67s 4s/step - loss: 29.6425 - accuracy: 0.5714 - val_loss: 29.6639 - val_accuracy: 0.5556
Epoch 6/50
16/16 [==============================] - 67s 4s/step - loss: 29.3254 - accuracy: 0.6190 - val_loss: 29.3155 - val_accuracy: 0.6667
Epoch 7/50
16/16 [==============================] - 66s 4s/step - loss: 28.7950 - accuracy: 0.6190 - val_loss: 28.9628 - val_accuracy: 0.8889
Epoch

In [13]:
model.save('D:/Model_Main/Xception_net_client_Snake_cnn2.h5')

In [14]:
# Fine-tuning Phase
base_model.trainable = True
for layer in base_model.layers[:300]:
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

history_fine = model.fit(
    AugmentedGenerator(train_generator),
    validation_data=val_generator,
    initial_epoch=history.epoch[-1],
    epochs=history.epoch[-1] + 30,
    
)

Epoch 50/79
16/16 [==============================] - 197s 10s/step - loss: 16.9401 - accuracy: 0.6190 - val_loss: 16.5371 - val_accuracy: 0.7778
Epoch 51/79
16/16 [==============================] - 150s 9s/step - loss: 16.8172 - accuracy: 0.6825 - val_loss: 16.2112 - val_accuracy: 0.8889
Epoch 52/79
16/16 [==============================] - 159s 10s/step - loss: 16.6971 - accuracy: 0.7143 - val_loss: 16.2254 - val_accuracy: 0.7778
Epoch 53/79
16/16 [==============================] - 157s 10s/step - loss: 16.3767 - accuracy: 0.8571 - val_loss: 16.3221 - val_accuracy: 0.7778
Epoch 54/79
16/16 [==============================] - 155s 10s/step - loss: 16.3987 - accuracy: 0.7778 - val_loss: 16.0024 - val_accuracy: 0.8889
Epoch 55/79
16/16 [==============================] - 153s 10s/step - loss: 16.5968 - accuracy: 0.7937 - val_loss: 16.1420 - val_accuracy: 0.8889
Epoch 56/79
16/16 [==============================] - 155s 10s/step - loss: 16.4880 - accuracy: 0.7937 - val_loss: 16.2015 - val_acc

In [15]:
model.save('D:/Model_Main/Xception_net_client_snake_cnn_tunned2.h5')

In [16]:
from tensorflow.keras.models import load_model

# Path to your saved model
model_path = 'D:/Model_Main/Xception_net_client_Snake_cnn2.h5'

# Load the model
model = load_model(model_path)

# Optional: Print model summary to confirm it's loaded correctly
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-l (Functiona  (None, 15, 15, 1280)     117746848 
 l)                                                              
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 batch_normalization (BatchN  (None, 1280)             5120      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 2048)              2623488   
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                        

In [17]:
# Evaluate the model
eval_results = model.evaluate(test_generator)
print(f"Test Loss: {eval_results[0]:.4f} | Test Accuracy: {eval_results[1]:.4f}")

5/5 [==============================] - 21s 3s/step - loss: 16.1888 - accuracy: 0.8889
Test Loss: 16.1888 | Test Accuracy: 0.8889


In [1]:
# Define class labels manually
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
import numpy as np
from tensorflow.keras.models import load_model

class_labels = {
    0: 'Bronzeback',
    1: 'Cat Snake',
    2: 'Central Pit viper',
    3: 'Kukri',
    4: 'Pit Viper',
    5: 'Plumbea',
    6: 'Red tail',
    7: 'Reticulated python',
    8: 'tytleri'
}

# Load the trained model
model = load_model('D:/Model_Main/Xception_net_client_snake_cnn_tunned2.h5')



In [39]:
# Load and preprocess the image
image_path = 'D:/Less_data_test/snake/s25.png'
test_image = load_img(image_path, target_size=(480, 480))  # Match model input size
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image = preprocess_input(test_image)  # Use EfficientNetV2 preprocessing

# Predict the class
result = model.predict(test_image)
predicted_class = np.argmax(result)

# Get the predicted label
prediction = class_labels[predicted_class]

print(f'The predicted class is: {prediction}')

1/1 [==============================] - 1s 1s/step
The predicted class is: tytleri
